## Plot WROMY data

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

from numpy import nan
from pathlib import Path
from obspy import UTCDateTime
from tqdm.notebook import tqdm_notebook
from datetime import datetime


from andbro__get_timeaxis import __get_timeaxis

### Methods

In [21]:
def __read_wromy_data(config, date):
    '''
    reads data from T1 to T2
    '''
    
    doy = str(date.timetuple().tm_yday).rjust(3,"0")

    path = f"{config['pathToData']}{date.year}/BW/WROMY/{config['channel']}.D/"

    if not Path(path).exists():
        __reply(f"Path: {path}, does not exists!")
        return


    fileName = f"BW.WROMY.{config['channel']}.D.{date.year}.{doy}"

#         print(f'   reading {fileName} ...')

    try:
        df0 = pd.read_csv(path+fileName)
        ## replace error indicating values (-9999, 999.9) with NaN values
        df0.replace(to_replace=-9999, value=nan, inplace=True)
        df0.replace(to_replace=999.9, value=nan, inplace=True)

#             ## change time from in to 6 character string
        df0.iloc[:,2] = [str(ttt).rjust(6,"0") for ttt in df0.iloc[:,2]]

    except:
        __reply(f"File: {fileName}, does not exists!")

    
    df0.reset_index(inplace=True, drop=True)

    ## add columns with total seconds
    
    if 'Seconds' in df0.columns:
        time_reference = datetime(2019,1,1)
        time_offset_seconds = (datetime(date.year,date.month,date.day) - time_reference).total_seconds()
        df0['totalSeconds'] = time_offset_seconds + df0['Seconds']
    
    
    return df0

### Configurations

In [22]:
config = {}

config['channel'] = "WS8"

config['tbeg'] = UTCDateTime("2022-08-28")
config['tend'] = UTCDateTime("2022-08-29")

config['date_range']  = pd.date_range(config['tbeg'].date, config['tend'].date)

config['pathToData'] = f"/import/freenas-ffb-01-data/romy_archive/"
config['pathToData'] = f"/home/andbro/Downloads/tmp/wromy/"

config['pathToOutput'] = f"/import/kilauea-data/wromy/{config['channel']}"
config['pathToOutput'] = f"/home/andbro/Downloads/tmp/wromy/"

config['resample'] = 600



In [23]:
config['channel'] = "WS8"


df = pd.DataFrame()
    
for n, date in enumerate(config['date_range']):    
                         
    df_data = __read_wromy_data(config, date)
    
    df_data['TimeStamp'] = pd.to_datetime(df_data['Date'].astype(str)+df_data['Time (UTC)'].astype(str))
    
    df_resampled = df_data.resample(f"{config['resample']/60}T", on="TimeStamp").mean()
    
    df_resampled.reset_index(inplace=True)
    
    df_resampled.pop("Date")
    df_resampled.pop("Seconds")

    filename = config['pathToOutput']+str(config['channel'])+"_"+str(date)[:10].replace("-","")
#     df_resampled.to_csv(filename+".csv")
    df_resampled.to_pickle(filename+".pkl")